<a href="https://colab.research.google.com/github/georgezefko/Anomaly-Detection-for-Sequential-Data/blob/main/VAE_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip3 install pickle5

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import numpy as np
import torch
from torch import nn, optim
from torch import distributions
from torch.utils.data import DataLoader, TensorDataset

from torch.autograd import Variable
import os
import numpy as np
from random import randint
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import copy
from torch.nn.parameter import Parameter
import math
from torch.nn import functional as F
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import pickle5 as pickle
import statsmodels.api
import os
from tqdm import tqdm
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters


sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 14, 10
register_matplotlib_converters()

batch_size = 64
green_mobility = True
max=False


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cuda = torch.cuda.is_available()


In [ ]:
def heartbeat(data_train,data_test_val):
  data = np.concatenate((data_train, data_test_val), axis=0)
  df = pd.DataFrame(data)
  new_columns = list(df.columns)
  new_columns[0] = 'target'
  df.columns = new_columns
  df.target = [int(i) for i in df.target]

  normal_df = df[df.target == 1].drop(labels='target', axis=1)
  anomaly_df = df[df.target != 1].drop(labels='target', axis=1)
  return normal_df,anomaly_df

In [ ]:

from scipy.signal import butter,filtfilt
# Filter requirements.
T = 5.0 # Sample Period
fs = 20.0 # sample rate, Hz
cutoff = 1.2 # desired cutoff frequency of the filter, Hz , slightly higher than actual 1.2 Hz
nyq = 0.5 * fs # Nyquist Frequency
order = 2 # sin wave can be approx represented as quadratic
n = int(T * fs) # total number of samples
 
def butter_lowpass_filter(data, cutoff, fs, order):
  normal_cutoff = cutoff / nyq
  # Get the filter coefficients 
  b, a = butter(order, normal_cutoff, btype='low', analog=False)
  y = filtfilt(b, a, data)
  return y
def smoothing(Normal,Anomaly): 
  Normal_smooth = pd.DataFrame()
  for i in range(Normal.shape[0]):
    Normal_smooth = Normal_smooth.append(pd.DataFrame(butter_lowpass_filter(Normal.loc[i].values, cutoff, fs, order)).transpose())
  Normal_smooth = Normal_smooth.reset_index(drop = True) 
 
  Anomaly_smooth = pd.DataFrame()
  for i in range(Anomaly.shape[0]):
    Anomaly_smooth = Anomaly_smooth.append(pd.DataFrame(butter_lowpass_filter(Anomaly.loc[i].values, cutoff, fs, order)).transpose())
  Anomaly_smooth = Anomaly_smooth.reset_index(drop = True)
  return Normal_smooth,Anomaly_smooth

In [ ]:
if green_mobility:
  path = '/content/drive/MyDrive/AdvancedML/data/resampled_df.pickle'
  with open(path,'rb') as fh:
      data = pickle.load(fh)
  data = data[['GM.acc.xyz.z_resampled','IRI_mean']]
  data_unstacked = data.iloc[:,0].apply(list).apply(pd.Series)
  df = pd.concat([data_unstacked,data['IRI_mean']],axis=1)
  normal_df = df[df['IRI_mean'] <= 2].drop(labels='IRI_mean',axis=1).reset_index(drop = True)
  anomaly_df = df[df['IRI_mean'] > 2].drop(labels='IRI_mean',axis=1).reset_index(drop = True)
else:
  data_train = np.loadtxt('/content/drive/MyDrive/AdvancedML/data/ECG5000/ECG5000_TRAIN', delimiter=',')
  data_test_val = np.loadtxt('/content/drive/MyDrive/AdvancedML/data/ECG5000/ECG5000_TEST', delimiter=',')[:-1]
  normal_df,anomaly_df = heartbeat(data_train,data_test_val)

#for smoothing(Needs correction) it has been used only for experimentation
#normal_df,anomaly_df=smoothing(normal_df,anomaly_df)

RANDOM_SEED = 42
train_df, val_df = train_test_split(Normal,test_size=0.33,random_state=RANDOM_SEED)
val_df, test_df = train_test_split(val_df,test_size=0.15, random_state=RANDOM_SEED)


# ## Scaling your dataset
 if max:
  scaler = MinMaxScaler(feature_range=(0, 1))
  train_df = scaler.fit_transform(train_df)
  test_df = scaler.transform(test_df)
  val_df = scaler.transform(val_df)
  anomaly_df = scaler.fit_transform(Anomaly)
 else:
  scaler = StandardScaler()
  train_df = scaler.fit_transform(train_df)
  test_df = scaler.transform(test_df)
  val_df = scaler.transform(val_df)
  anomaly_df = scaler.fit_transform(Anomaly)



train_df = np.expand_dims(train_df,-1)
test_df = np.expand_dims(test_df,-1)
val_df = np.expand_dims(val_df,-1)
anomaly_df = np.expand_dims(anomaly_df,-1)

seq_len = train_df.shape[1]
n_features = train_df.shape[2]

train_dataset = TensorDataset(torch.from_numpy(train_df).float())
test_dataset = TensorDataset(torch.from_numpy(test_df).float())
val_dataset = TensorDataset(torch.from_numpy(val_df).float())
anomaly_dataset = TensorDataset(torch.from_numpy(anomaly_df).float())



train_loader = DataLoader(train_dataset, batch_size=batch_size, pin_memory=cuda,drop_last=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, pin_memory=cuda,drop_last=True)
val_loader  = DataLoader(val_dataset, batch_size=batch_size, pin_memory=cuda, drop_last=True)
anomaly_loader = DataLoader(anomaly_dataset, batch_size=batch_size, pin_memory=cuda,drop_last=True)



In [ ]:
class Encoder(nn.Module):

  def __init__(self, n_features, hidden_size, hidden_layer_depth, latent_length, bidirectional,dropout):

    super(Encoder,self).__init__()


    self.n_features = n_features
    self.hidden_size = hidden_size
    self.hidden_layer_depth = hidden_layer_depth
    self.latent_length = latent_length
    self.bidirectional = bidirectional
    self.direction_factor = (2 if self.bidirectional else 1)
    


    self.rnn1 = nn.GRU( input_size=self.n_features, 
                       hidden_size=self.hidden_size, 
                       dropout = dropout, 
                       num_layers = self.hidden_layer_depth,
                       batch_first=True,bidirectional = self.bidirectional)
      
  def forward(self, x):
    
    x = x.to(device)
    batch = x.size(0)
    h0 = torch.zeros(hidden_layer_depth*self.direction_factor, batch, self.hidden_size).to(device)

    out,hidden_n = self.rnn1(x,h0) 
  
    if self.bidirectional:

      last_hidden_state = hidden_n.view(hidden_layer_depth, self.direction_factor, batch, self.hidden_size)[-1] #(num_directions, batch, hidden_size)
      
      h_1, h_2 = last_hidden_state[0], last_hidden_state[1]
      
      #X = h_1 + h_2                # Add both states (needs different input size for first linear layer)
      hidden = torch.cat((h_1, h_2), 1)  # Concatenate both states
      
    else:
      hidden = hidden_n[-1, :, :]
     
    

    return hidden



In [ ]:
class Latent(nn.Module):
 
    def __init__(self, hidden_size, latent_length,bidirectional):
        super(Latent, self).__init__()

        self.bidirectional = bidirectional
        self.direction_factor = (2 if self.bidirectional else 1)
        self.hidden_size = hidden_size
        self.latent_length = latent_length

        self.hidden_to_mean = nn.Linear(self.hidden_size*self.direction_factor, self.latent_length)
        self.hidden_to_logvar = nn.Linear(self.hidden_size*self.direction_factor, self.latent_length)

        nn.init.xavier_uniform_(self.hidden_to_mean.weight)
        nn.init.xavier_uniform_(self.hidden_to_logvar.weight)

    def forward(self, cell_output):
        
        self.latent_mean = self.hidden_to_mean(cell_output)
        self.latent_logvar = self.hidden_to_logvar(cell_output)
        z_mu = self.latent_mean
        z_var = self.latent_logvar
        

        
        std = torch.exp(0.5 * self.latent_logvar)
        eps = torch.randn_like(std)
        sample = eps.mul(std).add_(self.latent_mean)

        return sample, z_mu, z_var
        

In [ ]:

class Decoder(nn.Module):

  def __init__(self, seq_len, batch_size, hidden_size, hidden_layer_depth, latent_length, n_features,bidirectional):
    super(Decoder, self).__init__()

    self.hidden_size = hidden_size
    self.batch_size = batch_size
    self.seq_len = seq_len
    self.hidden_layer_depth = hidden_layer_depth
    self.latent_length = latent_length
    self.n_features = n_features
    self.bidirectional = bidirectional
    self.direction_factor = (2 if self.bidirectional else 1)
    
    

    self.rnn1 = nn.GRU(input_size=1,
                       hidden_size=self.hidden_size,
                       dropout = dropout,
                       num_layers=hidden_layer_depth,
                       batch_first=True,
                       bidirectional= bidirectional
                       )

    

    self.latent_to_hidden = nn.Linear(self.latent_length,self.hidden_size)
    self.hidden_to_output = nn.Linear(self.hidden_size*self.direction_factor, self.n_features)

    self.decoder_inputs = torch.zeros( self.batch_size, self.seq_len, 1, requires_grad=True).to(device)

    nn.init.xavier_uniform_(self.latent_to_hidden.weight)
    nn.init.xavier_uniform_(self.hidden_to_output.weight)


  def forward(self, x):
    
    x = x.to(device)
    h_state = self.latent_to_hidden(x)
    
    h_0 = torch.stack([h_state for _ in range(self.hidden_layer_depth*self.direction_factor)]).to(device)
    
    decoder_output, _ = self.rnn1(self.decoder_inputs, h_0)
    
    out = self.hidden_to_output(decoder_output)

    return out

In [ ]:
class VAE(nn.Module):

  def __init__(self, enc,lat,dec):

    super(VAE,self).__init__()

  
    self.enc = enc
    self.lat = lat
    self.dec = dec
    

  def forward(self, x):

    encoder_output = self.enc(x)
    sample,z_mu,z_var = self.lat(encoder_output)
    x_hat = self.dec(sample)



    return x_hat, z_mu, z_var 

In [ ]:
hidden_size = 256
hidden_layer_depth = 2
latent_length = 10
dropout = 0.5
bidirectional = True
# encoder
encoder = Encoder( n_features, hidden_size, hidden_layer_depth, latent_length,bidirectional, dropout)

#latent
latent = Latent( hidden_size, latent_length, bidirectional)

# decoder
decoder = Decoder(seq_len, batch_size, hidden_size, hidden_layer_depth, latent_length, n_features,bidirectional)

# vae
model = VAE(encoder,latent,decoder).to(device)



print(model)

VAE(
  (enc): Encoder(
    (rnn1): GRU(1, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  )
  (lat): Latent(
    (hidden_to_mean): Linear(in_features=512, out_features=10, bias=True)
    (hidden_to_logvar): Linear(in_features=512, out_features=10, bias=True)
  )
  (dec): Decoder(
    (rnn1): GRU(1, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
    (latent_to_hidden): Linear(in_features=10, out_features=256, bias=True)
    (hidden_to_output): Linear(in_features=512, out_features=1, bias=True)
  )
)


In [ ]:
def loss(X, x_hat, z_mu, z_var, criterion):
  

  if criterion == 'MSELoss':
    criterion = nn.MSELoss(size_average=False,reduction='sum')
  elif criterion == 'SmoothL1Loss':
    criterion = nn.SmoothL1Loss(size_average=False,reduction='sum')

  recon_loss = criterion(x_hat, X)
  KL_loss = -0.5 * torch.mean(1 + z_var - z_mu**2 - torch.exp(z_var))
  KL_loss = KL_loss/X.view(-1, X.size(1)).data.shape[0] * X.size(1)
  
  elbo_loss = recon_loss + KL_loss
  
  return elbo_loss, KL_loss

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.0006796395231964973) # lr derived from optimazation method

#optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [ ]:
#test of the model
X= next(iter(train_loader))
x = X[0].to(device)
outputs = model(x.float())


x_hat, z_mu, z_var = outputs

print('x',x.shape)
print('mu',z_mu.shape)
print('var',z_var.shape)
print('x_hat',x_hat.shape)



In [ ]:
#test on losses
elbo_loss, KL_loss = loss(x, x_hat, z_mu, z_var,criterion='MSELoss')

print(elbo_loss)
print(KL_loss)

In [ ]:
def train():
 

  model.train()

  train_loss = []
  train_kl = []
  
  

  for t, X in enumerate(train_loader):

    X =X[0]
    
    X = X.to(device)

    optimizer.zero_grad()

    x_hat, z_mu, z_var= model(X)

    elbo_loss, KL_loss = loss(X, x_hat, z_mu, z_var,criterion='MSELoss')
   

    elbo_loss.backward()

    train_loss.append(elbo_loss.item())
    train_kl.append(KL_loss.item())
    

    optimizer.step()
  
  train_elbo_loss = np.mean(train_loss)
  train_kl_loss = np.mean(train_kl)
 

  return  train_elbo_loss,train_kl_loss

In [ ]:
def test():

  model.eval()

  with torch.no_grad():
    test_loss = []
    test_kl = []
    
    for t, x in enumerate(val_loader):

      x = x[0]
      x = x.to(device)
      x_hat, z_mu, z_var = model(x)
      elbo_loss, KL_loss = loss(x, x_hat, z_mu, z_var,criterion='MSELoss')
      
      test_loss.append(elbo_loss.item())
      test_kl.append(KL_loss.item())
      

  test_elbo_loss = np.mean(test_loss)
  test_kl_loss = np.mean(test_kl)
  

  return model.eval(), test_elbo_loss,test_kl_loss

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
epochs = 100

best_model_wts = copy.deepcopy(model.state_dict())
best_loss = 10000.0
history_elbo = dict(train_elbo=[], test_elbo=[])
history_kl = dict(train_kl=[], test_kl=[])


device = torch.device("cuda:0" if cuda else "cpu")
print("Using device:", device)

for e in range(epochs):

  start_time = time.time()

  train_elbo_loss, train_kl_loss = train()
  model, test_elbo_loss, test_kl_loss  = test()

  end_time = time.time()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  if best_loss > test_elbo_loss:
    best_test_loss = test_elbo_loss
    best_model_wts = copy.deepcopy(model.state_dict())

  print(f'Epoch: {e+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\t train elbo loss {train_elbo_loss}   | train kl loss {train_kl_loss}') 
  print(f'\t val  val elbo loss {test_elbo_loss} | valid kl loss {test_kl_loss}') 


  history_elbo['train_elbo'].append(train_elbo_loss)
  history_elbo['test_elbo'].append(test_elbo_loss)

  history_kl['train_kl'].append(train_kl_loss)
  history_kl['test_kl'].append(test_kl_loss)


In [ ]:
# save the model
MODEL_PATH = 'model.pth'
torch.save(model, MODEL_PATH)

In [ ]:
fig = plt.figure(figsize=(20,15))


fig.add_subplot(2,2,1)
plt.plot(list(range(0,len(history_elbo['train_elbo']))), history_elbo['train_elbo'],'r')
plt.plot(list(range(0,len(history_elbo['test_elbo']))), history_elbo['test_elbo'],'g')
plt.legend(['Train ELBO','Val ELBO'])
plt.xlabel('EPOCH')
plt.ylabel('Loss')
plt.title('ELBO Loss')

fig.add_subplot(2,2, 2)
plt.plot(list(range(0,len(history_kl['train_kl']))), history_kl['train_kl'], 'r')
plt.plot(list(range(0,len(history_kl['test_kl']))), history_kl['test_kl'],'g')
plt.legend(['Train KL','Val KL'])
plt.xlabel('EPOCH')
plt.ylabel('Loss')
plt.title('KL Loss')

#plt.savefig('/content/drive/MyDrive/AdvancedML/data/gru_losses_bidirectional.png')


plt.show();



In [ ]:
def predict(model, dataset):
    predictions = []
    with torch.no_grad():
        model = model.eval()
    for t, X in enumerate(dataset):
        X = X[0]
        x = X.to(device)
        x_hat, z_mu, z_var = model(x)
        x_hat = x_hat.cpu().detach().numpy()
        x_hat = np.squeeze(x_hat, axis=2)
        predictions.append(x_hat)
    predictions = np.concatenate(predictions, axis=0, out= None)
    return predictions

In [ ]:
lillefor=True

In [ ]:
predict_train = predict(model, train_loader)
train_df = train_df[:predict_train.shape[0]]
train_original = np.squeeze(train_df,axis=2)

if lillefor:
  #Lillefors test on train
  train_residual = (train_original - predict_train)
  train_ksstat = np.zeros(train_residual.shape[0])
  for i in range(train_residual.shape[0]):
    train_ksstat[i] = statsmodels.stats.diagnostic.lilliefors(train_residual[i,], dist='norm', pvalmethod='table')[0]
  sns.distplot(train_ksstat, bins=40, kde=True)
  plt.show()
else:
  #MSE on train
  train_sq_loss = np.mean(( train_original- predict_train)**2, axis=1)
  sns.distplot(train_sq_loss, bins=40, kde=True)
  plt.show()

In [ ]:
predict_test = predict(model, test_loader)
test_df=test_df[:predict_test.shape[0]]
test_original = np.squeeze(test_df,axis=2)

if lillefor:
  #Lillefors test on test normal
  test_residual = (test_original - predict_test)
  test_ksstat = np.zeros(test_residual.shape[0])
  for i in range(test_residual.shape[0]):
    test_ksstat[i] = statsmodels.stats.diagnostic.lilliefors(test_residual[i,], dist='norm', pvalmethod='table')[0]
  sns.distplot(test_ksstat, bins=40, kde=True)
  #plt.savefig('/content/drive/MyDrive/AdvancedML/data/bigru_testnormal_rmse.png')
  plt.show()
else:
  #MSE on test normal
  test_sq_loss = np.mean(( test_original- predict_test)**2, axis=1)
  sns.distplot(test_sq_loss, bins=40, kde=True)
  plt.show()

In [ ]:
predict_anomaly = predict(model, anomaly_loader)
anomaly_df = anomaly_df[:predict_anomaly.shape[0]]

if lillefor:
  #Lillefors test on anomalies
  anomaly_residual = (np.squeeze(anomaly_df) - predict_anomaly)
  anomaly_ksstat = np.zeros(anomaly_residual.shape[0])
  for i in range(anomaly_residual.shape[0]):
    anomaly_ksstat[i] = statsmodels.stats.diagnostic.lilliefors(anomaly_residual[i,], dist='norm', pvalmethod='table')[0]

  sns.distplot(anomaly_ksstat, bins=20, kde=True)
  #plt.savefig('/content/drive/MyDrive/AdvancedML/data/bigru_anomaly_rmse.png')
  plt.show()
else:
  #MSE on anomalies
  anomaly_sq_loss = np.mean(( np.squeeze(anomaly_df) - predict_anomaly)**2, axis=1)
  sns.distplot(anomaly_sq_loss, bins=20, kde=True)
  plt.show()


In [ ]:
def plot_prediction(data, reconstructed, title, ax):
  ax.plot(data,'r', label='true')
  ax.plot(reconstructed, 'b',label='reconstructed')
  ax.set_title(f'{title} (Total loss: {np.around(np.mean(( data- reconstructed)**2), 2)})')
  ax.legend()

In [ ]:
#plot reconstrution on test normal and anomaly
fig, axs = plt.subplots(nrows=2,ncols=6,sharey=True,sharex=True,figsize=(22, 8))

for i, data in enumerate(test_original[0:6]):
  plot_prediction(data, predict_test[i], title='Normal', ax=axs[0, i])

for i, data in enumerate(anomaly_df[0:6]):
  plot_prediction(data, predict_anomaly[i], title='Anomaly', ax=axs[1, i])

fig.tight_layout();
#plt.savefig('/content/drive/MyDrive/AdvancedML/data/green_recon_smooth_vaebigru.png')


In [ ]:
#create classification dataframe with results from Lillefors test
label = np.concatenate((np.zeros(len(test_ksstat)),np.ones(len(anomaly_ksstat))))
#total_losses = np.concatenate((train_sq_loss,anomaly_sq_loss),axis=0)
total_ksstats = np.concatenate((test_ksstat,anomaly_ksstat),axis=0)
error_df_norm = pd.DataFrame({'ksstat': total_ksstats,'Label':label})
#error_df = error_df.sample(frac=1)
error_df.head(2)

In [ ]:
#ROC-AUC for Lillefors test
false_pos_rate, true_pos_rate, thresholds = roc_curve(error_df_norm.Label, error_df_norm.ksstat)
roc_auc = auc(false_pos_rate, true_pos_rate,)
plt.plot(false_pos_rate, true_pos_rate, linewidth=2, label='AUC = %0.3f'% roc_auc)
plt.plot([1,0],[1,0], linewidth=2)
plt.xlim([-0.01, 1])
plt.ylim([0, 1.01])
plt.legend(loc='lower right')
plt.title('Receiver operating characteristic curve (ROC)')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


In [ ]:
#create classification dataframe with results from MSE
label = np.concatenate((np.zeros(len(test_sq_loss)),np.ones(len(anomaly_sq_loss))))
#total_losses = np.concatenate((train_sq_loss,anomaly_sq_loss),axis=0)
total_losses = np.concatenate((test_sq_loss,anomaly_sq_loss),axis=0)
error_df = pd.DataFrame({'Reconstruction_error': total_losses,'Label':label})
#error_df = error_df.sample(frac=1)
error_df.head(2)

In [ ]:
#ROC-AUC for MSE
false_pos_rate, true_pos_rate, thresholds = roc_curve(error_df.Label, error_df.Reconstruction_error.values)
roc_auc = auc(false_pos_rate, true_pos_rate,)
plt.plot(false_pos_rate, true_pos_rate, linewidth=2, label='AUC = %0.3f'% roc_auc)
plt.plot([1,0],[1,0], linewidth=2)
plt.xlim([-0.01, 1])
plt.ylim([0, 1.01])
plt.legend(loc='lower right')
plt.title('Receiver operating characteristic curve (ROC)')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
#plt.savefig('/content/drive/MyDrive/adml/data/ROC_green.png')